In [21]:
import pandas as pd
from math import exp
from scipy.spatial import distance
from sklearn.metrics import roc_auc_score

In [22]:
def GradientDescent(C,data):
    print('C=%d' %C)
    errorAccuracy=10**-5
    weights=[0.,0.]
    k=0.1#длина шага
    l=data[0].count()#количество элементов в выборке
    distance_euclidean=0

    weightsDelta=[0.,0.]
    for step in range(10000):#Рекомендуется ограничить сверху число итераций десятью тысячами.
        oldweightsDelta=weightsDelta
        weightsDelta=[0.,0.]

        for obj in data.values:
            y=obj[0]
            gradient=y*(1-1/(1+exp(-y*(weights[0]*obj[1]+weights[1]*obj[2]))))
            weightsDelta=list(map(lambda w,wd,x: wd+x*gradient-k*C*w,weights,weightsDelta,obj[1:]))

        #доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5)
        distance_euclidean=distance.euclidean(weightsDelta,oldweightsDelta)
        if distance_euclidean<errorAccuracy:
            print('Дошли до заданной ошибки точности на шаге: %d' % step)
            break

        weights=list(map(lambda w,wd: w+wd*k/l,weights,weightsDelta))

    print('Евклидово расстояние=%.6f' %distance_euclidean)
    return weights
 

In [23]:
def GetAUC_ROC(C,data):
    weights=GradientDescent(C,data)
    y_true = list(map(lambda y: 0 if y<0 else 1 ,data[0].values))#Класс объекта
    X_train = data.loc[:, 1:]#Характеристики объектов

    y_scores=list(map(lambda x: 1 / (1 + exp(-weights[0]*x[0] - weights[1]*x[1])) ,X_train.values))
    aUC_ROC=roc_auc_score(y_true, y_scores)
    print ('AUC_ROC=%.3f' % round(aUC_ROC,3))
    print('-----')
    return round(aUC_ROC,3)



In [24]:
data= pd.read_csv('data-logistic.csv',header=None)
#выводим ответ
answ=[]
answ.append(GetAUC_ROC(0,data))
answ.append(GetAUC_ROC(2,data))

print ('Ответ')
print (' '.join(map(lambda x: str(x), answ)))

C=0
Дошли до заданной ошибки точности на шаге: 406
Евклидово расстояние=0.000010
AUC_ROC=0.927
-----
C=2
Дошли до заданной ошибки точности на шаге: 227
Евклидово расстояние=0.000010
AUC_ROC=0.934
-----
Ответ
0.927 0.934
